In [1]:
import numpy as np
import xarray as xr
import os
import pandas as pd
import glob

In [5]:
import sys ; sys.path.append("../..")
sys.path.append("../../ppe_tools/")
from ppe_tools import *
#from ../../ppe_tools/ import *

In [6]:
csv='lhc_220926.csv'

In [4]:
#data_url = '"https://docs.google.com/spreadsheets/d/e/2PACX-1vQLjMUT7sgunI4XkbGWq_RrjrLhR8-N0RiojM7h3lbSsN5-jgKIFyiJtPJeQkTEf2TL4ouLu_dqmEkg/pub?gid=737792973&single=true&output=csv"'
#cmd = 'curl -L '+data_url+' > '+csv # need to add -L option to force redirects
#os.system(cmd)

In [7]:
df = pd.read_csv(csv)
params=df['param'].values
params

array(['FUN_fracfixers', 'KCN', 'a_fix', 'crit_dayl', 'd_max', 'fff',
       'froot_leaf', 'fstor2tran', 'grperc', 'jmaxb0', 'jmaxb1', 'kcha',
       'kmax', 'krmax', 'leaf_long', 'leafcn', 'lmr_intercept_atkin',
       'lmrha', 'lmrhd', 'medlynintercept', 'medlynslope', 'nstem',
       'psi50', 'q10_mr', 'slatop', 'soilpsi_off', 'stem_leaf',
       'sucsat_sf', 'theta_cj', 'tpu25ratio', 'tpuse_sf', 'wc2wjb0'],
      dtype=object)

In [8]:
kcns=['kc_nonmyc',
        'kn_nonmyc',
        'akc_active',
        'akn_active',
        'ekc_active',
        'ekn_active']
kcns

['kc_nonmyc',
 'kn_nonmyc',
 'akc_active',
 'akn_active',
 'ekc_active',
 'ekn_active']

In [9]:
lhcs = {}
for p in params:
    ix       = df['param']==p
    minval   = df['min'][ix].values[0]
    maxval   = df['max'][ix].values[0]
    pftmin   = df['pft_mins'][ix].values[0]
    pftmax   = df['pft_maxs'][ix].values[0]
    thisloc  = df['loc'][ix].values[0]
    if p=='KCN':
        flag='KCN'
    else:
        flag=''
    
    needs_pft = (minval=='pft')
    if needs_pft:
        thismin = np.fromstring(pftmin, dtype='float', sep=',')
    elif 'percent' in minval:
        thismin = minval
    else:
        thismin = np.array(float(minval))

    needs_pft = (maxval=='pft')
    if needs_pft:
        thismax = np.fromstring(pftmax, dtype='float', sep=',')
    elif 'percent' in maxval:
        thismax = maxval
    else:
        thismax = np.array(float(maxval))

    if p!='KCN':
        lhcs[p]={'min':thismin,'max':thismax,'loc':thisloc,'flagged':[]}
    else:
        flagged={}
        for kcn in kcns:
            flagged[kcn]={'min':thismin,'max':thismax,'loc':thisloc}
        lhcs[p]={'min':[],'max':[],'loc':thisloc,'flagged':flagged}

### Start editing here

In [10]:
# read in parameter weights
# specify where your samples exist
# make sure that they are in the same order as the params in lhcs above
te = pd.read_csv('../../pyth/exp1_EmBE/exp1_EmBE_TotalError_psets.csv',index_col=0)
bm = pd.read_csv('../../pyth/exp1_EmBE/exp1_EmBE_BiomeSpecific_psets.csv',index_col=0)
psets = pd.concat([te,bm],ignore_index=True)

In [11]:
def get_p(p,i):
    return psets.iloc[i].loc[p]

In [12]:
exp1=[[get_p(p,i) for p in lhcs] for i in range(48)]

In [13]:
#should have 48 members and 32 param scale factors per member
np.shape(exp1)

(48, 32)

In [14]:
#instantiate the Ensemble object
basefile = '/glade/campaign/asp/djk2120/PPEn11/paramfiles/OAAT0000.nc'  #replace with /glade/campaign/asp/...
pdir = '/glade/scratch/djk2120/PPEn11/paramfiles/'
ndir = '/glade/scratch/djk2120/PPEn11/namelist_mods/'
x    = Ensemble(basefile,pdir,ndir)

#add the new LHCs
prefix = 'exp1_EmBE'   #make your own prefix
nextnum=1
n_samples=48  #fix this
x.add_lhcs(lhcs,prefix,nextnum,n_samples,lhc=exp1)

AttributeError: 'Ensemble' object has no attribute 'add_lhcs'

In [10]:
x.write(lhcfile='/glade/work/linnia/CLM-PPE-LAI_test/exp1_EmBE/psets_exp1_EBE_230419.txt')  #change to a file for your experiment

### testing stuff

In [20]:
from skopt.space import Space
from skopt.sampler import Lhs

In [21]:
n_samples=500
space = Space([(0.,1.) for p in params])
lhs = Lhs(lhs_type="classic", criterion=None)
lhc = lhs.generate(space.dimensions, n_samples)